In [ ]:
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile,ClassicalRegister,QuantumRegister
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import QasmSimulator
from qiskit.visualization import plot_histogram

In [ ]:
# Create the oracle circuit for a 3-qubit system.
# We want to mark the state |101>: 
#   - Qubit 0: desired value is 1 (no X needed)
#   - Qubit 1: desired value is 0 -> apply X to flip the condition.
#   - Qubit 2: desired value is 1 (no X needed)
#
# The trick: after applying an X on qubit 1, the only state satisfying |1 1 1>
# (on qubits 0,1,2) will be originally |1 0 1>. We then perform a controlled Z via
# a controlled-X sandwiched by H on the target.

oracle = QuantumCircuit(3, name='oracle')
oracle.x(1)             # Flip qubit 1 so that |101> -> |111>
oracle.h(2)             # Prepare to implement a controlled-Z via MCX on qubit2
oracle.mcx([0, 1], 2)     # Multi-controlled X gate (acts as controlled-Z when sandwiched with H)
oracle.h(2)             # Undo the previous Hadamard
oracle.x(1)             # Uncompute the X gate on qubit 1

# Optional: Visualize the oracle circuit
print("Oracle Circuit:")
print(oracle.draw())

# Create the GroverOperator using the defined oracle.
# The default state_preparation is H⊗n (i.e., Hadamard on each qubit)
grover_op = GroverOperator(oracle, insert_barriers=True)

# For insight, display the decomposition of the Grover operator.
print("\nGrover Operator (decomposed):")
print(grover_op.decompose().draw())

# Now, build the full circuit to run Grover's algorithm.
# 1. Prepare the initial state using Hadamard gates on all qubits.
# 2. Apply the Grover operator (one iteration).
# 3. Measure all qubits.
qc = QuantumCircuit(3, 3)
qc.h([0, 1, 2])                # State preparation: uniform superposition
qc.append(grover_op, [0, 1, 2])  # Grover iteration
qc.measure([0, 1, 2], [0, 1, 2]) # Measurement

print("\nFull Grover Circuit:")
print(qc.draw())

# Run the circuit on the Qiskit simulator to verify that the good state is amplified.
simulator = ('qasm_simulator')
result = execute(qc, backend=simulator, shots=1024).result()
counts = result.get_counts()

print("\nMeasurement Results:")
print(counts)